In [1]:
#Data pre-processing
#Import data from csv file
import csv
import numpy as np
import pandas as pd
from sklearn import svm
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import precision_recall_fscore_support
from sklearn.grid_search import GridSearchCV
from sklearn.model_selection import RepeatedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cross_validation import train_test_split
from sklearn.svm import SVC
#SVM 2 classes
#Create array matrix sizemxn where m is a number of data and n is a number of feature
vf_mat = np.zeros((121,53,))
vf_mat2 = np.zeros((121,55,))
with open('visual_field_pd.csv', newline='') as csvfile:
    vf_csv = csv.reader(csvfile)
    i = 0
    j = 0
    #Choose data by row
    for row in vf_csv:
        j=0
        #Choose single data in row
        for s in row:
            if int(row[0]) == 1:
                if j==0:
                    vf_mat[i,j] = 0
                else:
                    vf_mat[i,j] = int(s)
            elif int(row[0]) != 1:
                if j==0:
                    vf_mat[i,j] = 1
                else:
                    vf_mat[i,j] = int(s)                
            j=j+1
        i=i+1
        
with open('visual_field.csv', newline='') as csvfile:
    vf_csv = csv.reader(csvfile)
    i = 0
    j = 0
    #Choose data by row
    for row in vf_csv:
        j=0
        #Choose single data in row
        for s in row:
            vf_mat2[i,j] = int(s)
            j=j+1
        i=i+1

#Create array matrix of feature (Input)
X = vf_mat[:,1:]
X2 = vf_mat2[:,1:]

#Create array matrix of Output (Stage of Glaucoma 1 - 4)
y = vf_mat[:,0]
y2 = vf_mat2[:,0]-1
#Convert the Classification Report  into CSV file 
def pandas_classification_report(y_true, y_pred):
    metrics_summary = precision_recall_fscore_support(
            y_true=y_true, 
            y_pred=y_pred)
    
    avg = list(precision_recall_fscore_support(
            y_true=y_true, 
            y_pred=y_pred,
            average='weighted'))

    metrics_sum_index = ['precision', 'recall', 'f1-score', 'support']
    class_report_df = pd.DataFrame(
        list(metrics_summary),
        index=metrics_sum_index)

    support = class_report_df.loc['support']
    total = support.sum() 
    avg[-1] = total

    class_report_df['avg / total'] = avg
    return class_report_df.T

C:\Users\Maytus\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\Maytus\Anaconda3\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
#PCA
#Preprocessing data to standard variance
def pca_reduce_dim(X, dim):
    scaler = StandardScaler()
    scaler.fit(X)
    scaled_data = scaler.transform(X)
    #Applying PCA
    pca = PCA(n_components = dim)
    pca.fit(scaled_data)
    return(pca.transform(scaled_data))
X_pca = pca_reduce_dim(X2,2)

In [3]:
n_rep = 5
c = 1
rkf = RepeatedKFold(n_splits=5, n_repeats=n_rep, random_state=None)
y_test_dict = dict()
y_test_dict2class = dict()
pred_dict = dict()
pred_dict2class = dict()
confusion_dict = dict()
confusion_dict2 = dict()
for train_index, test_index in rkf.split(X):
    #SVM1
    #Classify into 2 classes i.e. normal eye (0) and Glaucoma (1)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y2[test_index]
    param_grid = {'C': [0.1,1,10,100,1000], 'gamma':[1,0.1,0.01,0.001,0.0001]}
    svc = GridSearchCV(SVC(), param_grid, verbose=1000)
    svc.fit(X_train, y_train)
    predictions = svc.predict(X_test)
    predictions2class = svc.predict(X_test)
    y_test2class = y[test_index]
    
    #SVM2
    #Classify Glaucoma into 3 stages (1-3)
    train_index2 = list()
    test_index2 = list()
    verify_index = list()
    for i in train_index:
        if y[i]!=0:
            train_index2.append(i)
    count_pred = 0
    for i in test_index:
        if predictions[count_pred] !=0:
            test_index2.append(i)
            verify_index.append(count_pred)
        count_pred+=1
    
    X2_train, X2_test = X_pca[train_index2], X_pca[test_index2]
    y2_train, y2_test = y2[train_index2], y2[test_index2]
    
    svc2 = GridSearchCV(SVC(), param_grid, verbose=1000)
    svc2.fit(X2_train, y2_train)
    predictions2 = svc2.predict(X2_test)
    
    predictions[verify_index] = predictions2
    #Keep y_test and predictions from each iteration in dict
    confusion_dict[str(c)] = pandas_classification_report(y_test,predictions)
    confusion_dict2[str(c)] = pandas_classification_report(y_test2class,predictions2class)
    
    #Keep y_test and predictions from each iteration in dict
    y_test_dict[str(c)] = y_test
    y_test_dict2class[str(c)] = y_test2class
    pred_dict[str(c)] = predictions
    pred_dict2class[str(c)] = predictions2class
    
    c+=1

Fitting 3 folds for each of 25 candidates, totalling 75 fits
[CV] C=0.1, gamma=1 ..................................................
[CV] ......................... C=0.1, gamma=1, score=0.718750 -   0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[CV] C=0.1, gamma=1 ..................................................
[CV] ......................... C=0.1, gamma=1, score=0.718750 -   0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[CV] C=0.1, gamma=1 ..................................................
[CV] ......................... C=0.1, gamma=1, score=0.718750 -   0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[CV] C=0.1, gamma=0.1 ................................................
[CV] ....................... C=0.1, gamma=0.1, score=0.718750 -   0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[CV] C=0.1, gamma=0.1 .............................

[CV] C=10, gamma=0.01 ................................................
[CV] ....................... C=10, gamma=0.01, score=1.000000 -   0.0s
[Parallel(n_jobs=1)]: Done  37 out of  37 | elapsed:    0.0s remaining:    0.0s
[CV] C=10, gamma=0.01 ................................................
[CV] ....................... C=10, gamma=0.01, score=0.913043 -   0.0s
[Parallel(n_jobs=1)]: Done  38 out of  38 | elapsed:    0.0s remaining:    0.0s
[CV] C=10, gamma=0.01 ................................................
[CV] ....................... C=10, gamma=0.01, score=0.913043 -   0.0s
[Parallel(n_jobs=1)]: Done  39 out of  39 | elapsed:    0.0s remaining:    0.0s
[CV] C=10, gamma=0.001 ...............................................
[CV] ...................... C=10, gamma=0.001, score=1.000000 -   0.0s
[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:    0.0s remaining:    0.0s
[CV] C=10, gamma=0.001 ...............................................
[CV] ...................... C=10, gamma=0

[Parallel(n_jobs=1)]: Done  54 out of  54 | elapsed:    0.0s remaining:    0.0s
[CV] C=100, gamma=0.001 ..............................................
[CV] ..................... C=100, gamma=0.001, score=0.727273 -   0.0s
[Parallel(n_jobs=1)]: Done  55 out of  55 | elapsed:    0.0s remaining:    0.0s
[CV] C=100, gamma=0.001 ..............................................
[CV] ..................... C=100, gamma=0.001, score=0.848485 -   0.0s
[Parallel(n_jobs=1)]: Done  56 out of  56 | elapsed:    0.0s remaining:    0.0s
[CV] C=100, gamma=0.001 ..............................................
[CV] ..................... C=100, gamma=0.001, score=0.903226 -   0.0s
[Parallel(n_jobs=1)]: Done  57 out of  57 | elapsed:    0.0s remaining:    0.0s
[CV] C=100, gamma=0.0001 .............................................
[CV] .................... C=100, gamma=0.0001, score=0.666667 -   0.0s
[Parallel(n_jobs=1)]: Done  58 out of  58 | elapsed:    0.0s remaining:    0.0s
[CV] C=100, gamma=0.0001 .......

[CV] ...................... C=1, gamma=0.0001, score=0.393939 -   0.0s
[Parallel(n_jobs=1)]: Done  28 out of  28 | elapsed:    0.0s remaining:    0.0s
[CV] C=1, gamma=0.0001 ...............................................
[CV] ...................... C=1, gamma=0.0001, score=0.718750 -   0.0s
[Parallel(n_jobs=1)]: Done  29 out of  29 | elapsed:    0.0s remaining:    0.0s
[CV] C=1, gamma=0.0001 ...............................................
[CV] ...................... C=1, gamma=0.0001, score=0.718750 -   0.0s
[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:    0.0s remaining:    0.0s
[CV] C=10, gamma=1 ...................................................
[CV] .......................... C=10, gamma=1, score=0.727273 -   0.0s
[Parallel(n_jobs=1)]: Done  31 out of  31 | elapsed:    0.0s remaining:    0.0s
[CV] C=10, gamma=1 ...................................................
[CV] .......................... C=10, gamma=1, score=0.718750 -   0.0s
[Parallel(n_jobs=1)]: Done  32 out of  32

[CV] ....................... C=0.1, gamma=0.1, score=0.727273 -   0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[CV] C=0.1, gamma=0.1 ................................................
[CV] ....................... C=0.1, gamma=0.1, score=0.718750 -   0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.0s remaining:    0.0s
[CV] C=0.1, gamma=0.1 ................................................
[CV] ....................... C=0.1, gamma=0.1, score=0.718750 -   0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    0.0s remaining:    0.0s
[CV] C=0.1, gamma=0.01 ...............................................
[CV] ...................... C=0.1, gamma=0.01, score=0.727273 -   0.0s
[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:    0.0s remaining:    0.0s
[CV] C=0.1, gamma=0.01 ...............................................
[CV] ...................... C=0.1, gamma=0.01, score=0.718750 -   0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8

...

[CV] ...................... C=0.1, gamma=0.01, score=0.696970 -   0.0s
[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:    0.0s remaining:    0.0s
[CV] C=0.1, gamma=0.01 ...............................................
[CV] ...................... C=0.1, gamma=0.01, score=0.687500 -   0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.0s remaining:    0.0s
[CV] C=0.1, gamma=0.01 ...............................................
[CV] ...................... C=0.1, gamma=0.01, score=0.687500 -   0.0s
[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    0.0s remaining:    0.0s
[CV] C=0.1, gamma=0.001 ..............................................
[CV] ..................... C=0.1, gamma=0.001, score=0.757576 -   0.0s
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s remaining:    0.0s
[CV] C=0.1, gamma=0.001 ..............................................
[CV] ..................... C=0.1, gamma=0.001, score=0.687500 -   0.0s
[Parallel(n_jobs=1)]: Done  11 out of  11

[CV] ........................ C=1000, gamma=1, score=0.869565 -   0.0s
[Parallel(n_jobs=1)]: Done  61 out of  61 | elapsed:    0.0s remaining:    0.0s
[CV] C=1000, gamma=1 .................................................
[CV] ........................ C=1000, gamma=1, score=0.913043 -   0.0s
[Parallel(n_jobs=1)]: Done  62 out of  62 | elapsed:    0.0s remaining:    0.0s
[CV] C=1000, gamma=1 .................................................
[CV] ........................ C=1000, gamma=1, score=0.809524 -   0.0s
[Parallel(n_jobs=1)]: Done  63 out of  63 | elapsed:    0.0s remaining:    0.0s
[CV] C=1000, gamma=0.1 ...............................................
[CV] ...................... C=1000, gamma=0.1, score=0.913043 -   0.0s
[Parallel(n_jobs=1)]: Done  64 out of  64 | elapsed:    0.0s remaining:    0.0s
[CV] C=1000, gamma=0.1 ...............................................
[CV] ...................... C=1000, gamma=0.1, score=0.956522 -   0.0s
[Parallel(n_jobs=1)]: Done  65 out of  65

[CV] ........................ C=10, gamma=0.1, score=1.000000 -   0.0s
[Parallel(n_jobs=1)]: Done  34 out of  34 | elapsed:    0.0s remaining:    0.0s
[CV] C=10, gamma=0.1 .................................................
[CV] ........................ C=10, gamma=0.1, score=1.000000 -   0.0s
[Parallel(n_jobs=1)]: Done  35 out of  35 | elapsed:    0.0s remaining:    0.0s
[CV] C=10, gamma=0.1 .................................................
[CV] ........................ C=10, gamma=0.1, score=0.956522 -   0.0s
[Parallel(n_jobs=1)]: Done  36 out of  36 | elapsed:    0.0s remaining:    0.0s
[CV] C=10, gamma=0.01 ................................................
[CV] ....................... C=10, gamma=0.01, score=1.000000 -   0.0s
[Parallel(n_jobs=1)]: Done  37 out of  37 | elapsed:    0.0s remaining:    0.0s
[CV] C=10, gamma=0.01 ................................................
[CV] ....................... C=10, gamma=0.01, score=1.000000 -   0.0s
[Parallel(n_jobs=1)]: Done  38 out of  38

[CV] ..................... C=1000, gamma=0.01, score=0.875000 -   0.0s
[Parallel(n_jobs=1)]: Done  69 out of  69 | elapsed:    0.0s remaining:    0.0s
[CV] C=1000, gamma=0.001 .............................................
[CV] .................... C=1000, gamma=0.001, score=0.848485 -   0.0s
[Parallel(n_jobs=1)]: Done  70 out of  70 | elapsed:    0.0s remaining:    0.0s
[CV] C=1000, gamma=0.001 .............................................
[CV] .................... C=1000, gamma=0.001, score=0.875000 -   0.0s
[Parallel(n_jobs=1)]: Done  71 out of  71 | elapsed:    0.0s remaining:    0.0s
[CV] C=1000, gamma=0.001 .............................................
[CV] .................... C=1000, gamma=0.001, score=0.906250 -   0.0s
[Parallel(n_jobs=1)]: Done  72 out of  72 | elapsed:    0.0s remaining:    0.0s
[CV] C=1000, gamma=0.0001 ............................................
[CV] ................... C=1000, gamma=0.0001, score=0.757576 -   0.0s
[Parallel(n_jobs=1)]: Done  73 out of  73

[CV] ...................... C=10, gamma=0.001, score=0.848485 -   0.0s
[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:    0.0s remaining:    0.0s
[CV] C=10, gamma=0.001 ...............................................
[CV] ...................... C=10, gamma=0.001, score=0.939394 -   0.0s
[Parallel(n_jobs=1)]: Done  41 out of  41 | elapsed:    0.0s remaining:    0.0s
[CV] C=10, gamma=0.001 ...............................................
[CV] ...................... C=10, gamma=0.001, score=0.967742 -   0.0s
[Parallel(n_jobs=1)]: Done  42 out of  42 | elapsed:    0.0s remaining:    0.0s
[CV] C=10, gamma=0.0001 ..............................................
[CV] ..................... C=10, gamma=0.0001, score=0.727273 -   0.0s
[Parallel(n_jobs=1)]: Done  43 out of  43 | elapsed:    0.0s remaining:    0.0s
[CV] C=10, gamma=0.0001 ..............................................
[CV] ..................... C=10, gamma=0.0001, score=0.909091 -   0.0s
[Parallel(n_jobs=1)]: Done  44 out of  44

In [4]:
predictions

array([1., 1., 0., 1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 2.,
       2., 2., 2., 2., 2., 3., 3.])

In [5]:
y_test

array([0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 2.,
       2., 2., 2., 2., 2., 3., 3.])

In [6]:
confusion_dict

{'1':              precision    recall  f1-score  support
 0             1.000000  0.714286  0.833333      7.0
 1             0.818182  0.900000  0.857143     10.0
 2             0.800000  1.000000  0.888889      4.0
 3             1.000000  1.000000  1.000000      4.0
 avg / total   0.895273  0.880000  0.878413     25.0,
 '10':              precision  recall  f1-score  support
 0              0.75000   0.600  0.666667      5.0
 1              0.60000   0.750  0.666667      4.0
 2              1.00000   1.000  1.000000      5.0
 3              1.00000   1.000  1.000000     10.0
 avg / total    0.88125   0.875  0.875000     24.0,
 '11':              precision  recall  f1-score  support
 0             1.000000   0.875  0.933333      8.0
 1             0.857143   1.000  0.923077      6.0
 2             1.000000   1.000  1.000000      8.0
 3             1.000000   1.000  1.000000      3.0
 avg / total   0.965714   0.960  0.960205     25.0,
 '12':              precision    recall  f1-score 

In [7]:
pred_total = np.array([])
y_test_total = np.array([])
pred_total2class = np.array([])
y_test_total2class = np.array([])
for i in y_test_dict.values():
    y_test_total = np.concatenate((y_test_total,i),axis=0)
for i in pred_dict.values():
    pred_total = np.concatenate((pred_total,i),axis=0)
for i in y_test_dict2class.values():
    y_test_total2class = np.concatenate((y_test_total2class,i),axis=0)
for i in pred_dict2class.values():
    pred_total2class= np.concatenate((pred_total2class,i),axis=0)

In [8]:
#Classification
#0 = Normal Eye
#1 = Mild Glaucoma
#2 = Moderate Glaucoma
#3 = Advance Glaucoma
pandas_classification_report(y_test_total, pred_total)

,precision,recall,f1-score,support
0,0.925926,0.735294,0.819672,170.0
1,0.783410,0.918919,0.845771,185.0
2,0.956250,0.987097,0.971429,155.0
3,1.000000,0.978947,0.989362,95.0
avg / total,0.901747,0.894215,0.893178,605.0


In [9]:
#Detection
#0 = Normal Eye
#1 = Glaucoma
pandas_classification_report(y_test_total2class, pred_total2class)

,precision,recall,f1-score,support
0,0.925926,0.735294,0.819672,170.0
1,0.904255,0.977011,0.939227,435.0
avg / total,0.910345,0.909091,0.905633,605.0
